# HMM Model

In [ ]:
"""
A matrix:

            healthy    depressed
healthy      0.8          0.2

depressed   0.001         0.999


pi matrix:
            
healthy  depressed
 0.5        0.5
 
 
B matrix:

            movement   passive net   active net   texting   visiting
healthy       0.5        0.35          0.2          0.2       0.2

depressed     0.5        0.1           0.3          0.1       0.0


Observable state: [0, 1, 2, 3, 4]

"""

In [16]:
import numpy as np

PI = np.array( [0.5, 0.5] )

A = np.array([ [0.8, 0.2],
                [0.001, 0.999] ])

B = np.array([ [0.5, 0.35, 0.2, 0.2, 0.2],
               [0.5, 0.1, 0.3, 0.1, 0.0]  ])

Observables =  np.array( [0, 1, 2, 3, 4] )



#  Filtering
### Use the forward algorithm to compute the posterior distribution over the most recent state given all evidence (observation) to date.

In [17]:
def forward(PI, A, B, Observables):
    M = len(Observables) # nber of observation symbols
    N = A.shape[0]
    
    alpha = np.zeros( ( N, M) )
    alpha[ :, 0] = PI * B[ :, Observables[0] ]
#     print(alpha)
    
    for t in range(1, M):
        for j in range(N):
            for i in range(N):
                alpha[j, t] += alpha[i, t-1] * A[i, j] * B[j, Observables[t]] 
                
    return alpha

In [18]:
forward(PI, A, B, Observables)

array([[0.25      , 0.0700875 , 0.01122   , 0.00179784, 0.00028796],
       [0.25      , 0.029975  , 0.01318876, 0.00154196, 0.        ]])

### Implement the normalized forward algorithm

In [19]:
def forward_normalized(PI, A, B, Observables):
    M = len(Observables) # nber of observation symbols
    N = A.shape[0]
    
    alpha = np.zeros( ( N, M) )
    alpha[ :, 0] = PI * B[ :, Observables[0] ] # initialize alpha[:,0]

    c = np.zeros(M)
    c[0] = sum(alpha[:, 0]) # compute c[0] simple have alpha[:,0]
    
    # compute alpha
    for t in range(1, M):
        for j in range(N):
            for i in range(N):
                alpha[j, t] += alpha[i, t-1] * A[i, j] * B[j, Observables[t]] 
            
        c[t] =  sum(alpha[ :, t])
    
    # compute alpha_normalized
    alpha_normalized = np.zeros( (N, M) )
    for x in range(N):
        for y in range(M):
            alpha_normalized[x, y] = alpha[x, y] / c[y]  
            
    return alpha_normalized, c

In [20]:
forward_normalized(PI, A, B, Observables)

(array([[0.5       , 0.70043723, 0.45967097, 0.53830778, 1.        ],
        [0.5       , 0.29956277, 0.54032903, 0.46169222, 0.        ]]),
 array([5.00000000e-01, 1.00062500e-01, 2.44087525e-02, 3.33979373e-03,
        2.87962304e-04]))

## Evaluation

### Implement the following function to evaluate the probability of a given observation sequence by using the unnormalized forward variable

In [21]:
alpha = forward(PI, A, B, Observables)

def evaluation_unnormalized(alpha):
    
    evaluation = 0
    for y in range(alpha.shape[1]):
        evaluation +=  np.sum(alpha[:, y]) 
        
    return evaluation 

In [23]:
evaluation_unnormalized(alpha)

0.6280990085293447

### Evaluate the log probability of the given observation sequence by using the normalized forward variables

In [24]:
c = forward_normalized(PI, A, B, Observables)[1]

def evaluation_normalized(c):
    
    evaluation_nor = 0
    for t in range(c.shape[0]):
        evaluation_nor += np.log( c[t] )
        
    return -evaluation_nor

In [25]:
evaluation_normalized(c)

20.562448180121468

## Backward Probability

### Using the backward algorithm compute the backward variables, which is the probability of being in state Si at time t and observing the partial sequence Ot+1, Ot+2, …, OT.

In [26]:
def backward(PI, A, B, Observables):
    
    N = A.shape[0]
    M =len(Observables)
    
    beta = np.zeros( (N, M )  ) # N*M matrix
    beta[:, M-1] = 1 # initialize beta_T
    
    for t in range( M-2, -1, -1): # stop at T-1
        for j in range(N):
            for i in range(N):
                beta[i,t] += B[ j, Observables[t+1] ] * beta[ j, t+1 ] * A[i,j]
                
    return beta
        

In [27]:
backward(PI, A, B, Observables)

array([[1.14834645e-03, 4.09975880e-03, 2.56040000e-02, 1.60000000e-01,
        1.00000000e+00],
       [3.50276626e-06, 2.06992060e-05, 5.19800000e-05, 2.00000000e-04,
        1.00000000e+00]])

### Implement the normalized backward algorithm

In [28]:
# c = forward_normalized(PI, A, B, Observables)[1] # c for forward normalized

def backward_normalized(PI, A, B, c, Observables):
    
    N = A.shape[0]
    M =len(Observables)
    
    beta = np.zeros( (N, M )  ) # N*M matrix
    beta[:, M-1] = 1 # initialize beta_T
    
    beta_nor = np.zeros( (N,M) )
    
    for t in range( M-2, -1, -1): # stop at T-1
        for j in range(N):
            for i in range(N):
                beta[i,t] += B[ j, Observables[t+1] ] * beta[ j, t+1 ] * A[i,j]
    
    for x in range(N):
        for y in range(M):
            beta_nor[x,y] = beta[x,y] / c[y]
                
    return beta_nor

In [29]:
backward_normalized(PI, A, B, c, Observables)

array([[2.29669290e-03, 4.09719805e-02, 1.04896799e+00, 4.79071503e+01,
        3.47267676e+03],
       [7.00553252e-06, 2.06862771e-04, 2.12956398e-03, 5.98839379e-02,
        3.47267676e+03]])

### Smoothing by Forward-Backward Algorithm

In [30]:
alpha_normalized = forward_normalized(PI, A, B, Observables)[0]
beta_normalized = backward_normalized(PI, A, B, c, Observables)
noOfStates = A.shape[0]
noOfTimeSteps = B.shape[1]

def smoothed_probability(alpha_normalized, beta_normalized, noOfStates, noOfTimeSteps):
    
    smoothed_probability = np.zeros( (noOfStates,noOfTimeSteps) )
    
    numerator = 0
    denominator = 0
    for t in range(noOfTimeSteps):
        
        for i in range(noOfStates):
            numerator = alpha_normalized[i, t] * beta_normalized[i,t]
            
            for j in range(noOfStates):
                denominator += alpha_normalized[j, t] * beta_normalized[j,t] 
            
            smoothed_probability[i,t] = numerator / denominator
            
    return smoothed_probability

In [31]:
smoothed_probability(alpha_normalized, beta_normalized, noOfStates, noOfTimeSteps)

array([[9.96959006e-01, 9.23845309e-01, 8.87739417e-01, 9.60729549e-01,
        9.85062597e-01],
       [1.52049687e-03, 1.03584081e-03, 1.12097526e-03, 5.25033827e-04,
        0.00000000e+00]])